<a href="https://colab.research.google.com/github/freyzamarshall02/WebConsoleForMinecraftServers/blob/main/WebConsoleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check Location
import requests
import codecs
from IPython.display import display, HTML

# Get the public IP address
public_ip = requests.get('https://api.ipify.org').text
print(f"Public IP Address: {public_ip}")

# Get the geolocation information of the IP address
response = requests.get(f'https://ipinfo.io/{public_ip}/json')
data = response.json()

# Print the geolocation information
print(f"IP Address: {data['ip']}")
print(f"City: {data['city']}")
print(f"Region: {data['region']}")
print(f"Country: {data['country']}")
print(f"Location: {data['loc']}")
print(f"Organization: {data['org']}")
print(f"Timezone: {data['timezone']}")

# Check if the country is Taiwan
if data['country'] == 'TW':  # 'TW' is the country code for Taiwan
    display(HTML('<p style="color: green; font-weight: bold;">Your Server Good</p>'))
else:
    display(HTML('<p style="color: red; font-weight: bold;">The server may be bad</p>'))


In [ ]:
#@title Mount Drive and Clone Repo
from IPython.display import clear_output, Javascript
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/taifu9920/Web-based-Minecraft-server-control.git
clear_output()
print('\033[32m\033[1m\033[3mDone..\033[0m')

In [ ]:
#@title Setup
#@title Install Kebutuhan
import subprocess
import os
import requests
import threading
from IPython.display import clear_output, Javascript

installation = requests.get("https://pastebin.com/raw/Ak5wm7Yp").text
exec(installation)

# FileBrowserWeb
!curl -fsSL https://raw.githubusercontent.com/filebrowser/get/master/get.sh | bash

# Dependencies
%cd /content/Web-based-Minecraft-server-control/Setup
!pip install pyngrok -q
!chmod +x setup.sh
!./setup.sh
%cd /content/Web-based-Minecraft-server-control
!chmod +x main.py
!rm /content/Web-based-Minecraft-server-control/config.py
!wget -q -O config.py "https://huggingface.co/freyza/config/resolve/main/config.py"
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod a+x cloudflared
clear_output()
print('\033[32m\033[1m\033[3mDone..\033[0m')

In [ ]:
#@title Run App
#@markdown **Select one of the tunnels**

TUNNEL = "Cloudflared"  #@param ["Cloudflared", "NGROK"]

#@markdown ---
#@markdown Please Input Your Token (For NGROK Only)
Token = ""  #@param {type:"string"}

import os
import time
import urllib.request
import nest_asyncio
nest_asyncio.apply()
import ipywidgets as widgets
from IPython.display import display, Javascript
from pyngrok import ngrok
import subprocess

# Start filebrowser in the background
%cd /content
process = subprocess.Popen(["filebrowser", "-r", "/content/drive/MyDrive", "--port", "9876"])
# The process will keep running in the background.
print(f"Filebrowser is running with PID: {process.pid}")

%cd /content/Web-based-Minecraft-server-control

# ~~~~~~~~~~~~~~~~~~~~~~~~~~

if TUNNEL == "NGROK":
    # Ngrok Tunnels
    Close_Ngrok = True
    Open_New_Tab = False

    if Close_Ngrok:
        ngrok.kill()

    ngrok.set_auth_token(Token)

    console_tunnel = ngrok.connect(7878)
    file_tunnel = ngrok.connect(9876)

    clear_output()

    if Open_New_Tab:
        display(Javascript('window.open("{url}", "_blank");'.format(url=console_tunnel.public_url)))
        display(Javascript('window.open("{url}", "_blank");'.format(url=file_tunnel.public_url)))

    print("Console URL:", console_tunnel.public_url)
    print("File URL:", file_tunnel.public_url)

    # Start App
    !./main.py

elif TUNNEL == "Cloudflared":
    # Cloudflared
    # Clear the recently URL logs
    !rm /content/Web-based-Minecraft-server-control/Console_Url
    !rm /content/Web-based-Minecraft-server-control/FileBrowser_Url

    print("Starting Cloudflared Tunnel for Console")
    f1 = open("Console_Url", "w")
    p1 = subprocess.Popen(['./cloudflared', '--url', 'http://localhost:7878'], bufsize=0, stdout=f1, stderr=subprocess.STDOUT)

    print("Starting Cloudflared Tunnel for FileBrowser")
    f2 = open("FileBrowser_Url", "w")
    p2 = subprocess.Popen(['./cloudflared', '--url', 'http://localhost:9876'], bufsize=0, stdout=f2, stderr=subprocess.STDOUT)

    time.sleep(6)

    # Search for "trycloudflare" in both stdout files
    print("--------- Console URL ---------")
    !grep -F trycloudflare Console_Url
    print("---------------------------------")
    print("--------- File URL ---------")
    !grep -F trycloudflare FileBrowser_Url
    print("---------------------------------")

    # Start App
    !./main.py

    # Stop Cloudflare
    p1.terminate()
    p2.terminate()
# ~~~~~~~~~~~~~~~~~~~~~~~~~~

# Kill Filemanager App If The It Doesn't Stop Correctly
!kill -9 $(lsof -t -i:9876)
clear_output()
print("--------- SERVER STOPPED! ---------")